In [1]:
import random
import uuid
from collections import Counter

from typed_lists import TypedList, _initialize_global_thread_pool

## Examples

In [2]:
# can set number of threads to use for parallel operations
# this is a global setting and can only be set once
# calling this function again will raise an error

_initialize_global_thread_pool(4)

In [3]:
# examples of creating a typed list 
print(TypedList(range(10)) * TypedList(range(10)))
print(TypedList(["a", "b", "c", "d", "e"]) + TypedList(["f", "g", "h", "i", "j"]))
print(TypedList([1.0, 2.0, 3.0, 4.0, 5.0]) / TypedList([1.0, 2.0, 3.0, 4.0, 5.0]))
print(TypedList([bool(i%2) for i in range(10)], "bool"))

IntTypedList([0, 1, 4, 9, 16]...)
StringTypedList (["af", "bg", "ch", "di", "ej"])
FloatTypedList ([1.0, 1.0, 1.0, 1.0, 1.0])
BoolTypedList([false, true, false, true, false]...)


In [4]:
# parallelized sorting
uuids = TypedList([str(uuid.uuid4()) for _ in range(1_000_000)])
uuids.sort()
uuids

StringTypedList(["00000cf1-870c-469d-99d5-7343782ba814", "00002f02-9b33-476c-91fd-557c9ceedea6", "0000334f-96ef-46ff-a895-9a6abc9c95d9", "0000592a-f9e5-4b00-beb9-bf8546ca8b15", "0000612a-0b99-4565-b199-e43dcb63bd92"]...)

In [5]:
# parallelized counting
ids = TypedList([random.randint(0, 4) for _ in range(1_000_000)])
ids.count()

{0: 200064, 2: 200951, 3: 199595, 1: 199838, 4: 199552}

In [6]:
# broadcasting and parallelized operations
a = TypedList(range(1_000_000))
b = TypedList(range(1_000_000))
a + b * 3

IntTypedList([0, 4, 8, 12, 16]...)

In [7]:
# slice access
print(TypedList(range(10))[1:5])

# slice assignement
my_typed_list = TypedList(range(10))
my_typed_list[:4] = TypedList(range(4, 0, -1))
print(my_typed_list)

IntTypedList ([1, 2, 3, 4])
IntTypedList([4, 3, 2, 1, 4]...)


In [8]:
# for bool you can use the &, |, and ^ operators
# and they're parallelized and broadcasted
# Note: __bool__ still corresponds to the length of the list

# fizzbuzz example
fizz = TypedList([i%3 == 0 and i > 1 for i in range(1_000)])
buzz = TypedList([i%5 == 0 and i > 1 for i in range(1_000)])
print(f"fizz buzz: {(fizz & buzz).count()}")

fizz buzz: {True: 66, False: 934}


In [9]:
# comparing typed lists always generates a BoolTypedList 
print(TypedList(range(10)) == 2)
print(TypedList(range(10)) == TypedList(range(10)))

# BoolTypedLists can be collapsed to a single bool w/ any or all
print((TypedList(range(10)) == 2).any())

# BoolTypedLists can be used as a mask
print(
    TypedList(range(10))[TypedList([bool(i%2) for i in range(10)])]
)

BoolTypedList([false, false, true, false, false]...)
BoolTypedList([true, true, true, true, true]...)
True
IntTypedList ([1, 3, 5, 7, 9])


## Performance Example

In [10]:
# count "random" strings
rust_strs = TypedList([str(random.randint(1, 100)) for _ in range(100)] * 10_000)
py_strs = rust_strs.data

In [11]:
%%timeit
_ = rust_strs.count()

14.5 ms ± 452 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%%timeit
_ = Counter(py_strs)

36.4 ms ± 188 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
# faster using 4 threads
assert rust_strs.count() == Counter(py_strs)